In [1]:
import ccxt
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import tz
import matplotlib.pyplot as plt

import json
import time
import os
import sys

In [2]:
src_path = '../src/'
sys.path.append(os.path.abspath(src_path))
from func_get import *
from func_signal import *
from func_backtest import *

In [3]:
with open('../../../_keys/ftx/ftx_read_keys.json') as keys_file:
    keys_dict = json.load(keys_file)
    
exchange = ccxt.ftx({'apiKey': keys_dict['apiKey'],
                     'secret': keys_dict['secret'],
                     'enableRateLimit': True})

### Config

In [4]:
symbol = 'ETH-PERP'
timeframe = '15m'

### Get data

In [5]:
ohlcv_df = get_ohlcv_df(exchange, symbol, timeframe, since=None, limit=1000)
ohlcv_df = ohlcv_df.loc[:len(ohlcv_df) - 2, :]

In [6]:
ohlcv_df

,time,open,high,low,close,volume
0,2022-02-21 12:15:00,2740.5,2756.7,2738.1,2747.9,1.322385e+07
1,2022-02-21 12:30:00,2747.9,2748.6,2738.8,2740.3,1.196507e+07
2,2022-02-21 12:45:00,2740.3,2743.1,2734.8,2738.0,9.892660e+06
3,2022-02-21 13:00:00,2738.0,2741.1,2728.7,2736.8,1.480523e+07
4,2022-02-21 13:15:00,2736.8,2741.4,2734.3,2741.4,3.978329e+06
...,...,...,...,...,...,...
994,2022-03-03 20:45:00,2924.5,2932.1,2917.2,2930.6,2.687443e+07
995,2022-03-03 21:00:00,2930.6,2935.0,2919.2,2924.3,3.722915e+07
996,2022-03-03 21:15:00,2924.3,2925.9,2915.7,2918.6,1.243996e+07
997,2022-03-03 21:30:00,2918.7,2922.4,2883.2,2891.8,4.045452e+07


### Signal

### Donchian

In [7]:
def add_donchian(ohlcv_df):
    def get_donchian_side(ohlcv_df):
        if ohlcv_df['close'] > ohlcv_df['max_high']:
            signal_side = 'buy'
        elif ohlcv_df['close'] < ohlcv_df['min_low']:
            signal_side = 'sell'
        else:
            signal_side = np.nan

        return signal_side
    
    
    signal_dict = {
        'windows': 30,
        'revert': False
    }
    
    windows = signal_dict['windows']
    
    temp_df = ohlcv_df.copy()

    max_high_list = temp_df['high'].rolling(window=signal_dict['windows']).max()
    min_low_list = temp_df['low'].rolling(window=signal_dict['windows']).min()
    
    temp_df['max_high'] = max_high_list
    temp_df['min_low'] = min_low_list
    
    temp_df['max_high'] = temp_df['max_high'].shift(periods=1)
    temp_df['min_low'] = temp_df['min_low'].shift(periods=1)
    temp_df['donchian'] = temp_df.apply(get_donchian_side, axis=1)
    
    donchian_list = temp_df['donchian'].fillna(method='ffill')
    ohlcv_df['donchian'] = donchian_list
    
    return ohlcv_df

In [8]:
test_df = add_donchian(ohlcv_df)

In [9]:
test_df

,time,open,high,low,close,volume,donchian
0,2022-02-21 12:15:00,2740.5,2756.7,2738.1,2747.9,1.322385e+07,NaN
1,2022-02-21 12:30:00,2747.9,2748.6,2738.8,2740.3,1.196507e+07,NaN
2,2022-02-21 12:45:00,2740.3,2743.1,2734.8,2738.0,9.892660e+06,NaN
3,2022-02-21 13:00:00,2738.0,2741.1,2728.7,2736.8,1.480523e+07,NaN
4,2022-02-21 13:15:00,2736.8,2741.4,2734.3,2741.4,3.978329e+06,NaN
...,...,...,...,...,...,...,...
994,2022-03-03 20:45:00,2924.5,2932.1,2917.2,2930.6,2.687443e+07,buy
995,2022-03-03 21:00:00,2930.6,2935.0,2919.2,2924.3,3.722915e+07,buy
996,2022-03-03 21:15:00,2924.3,2925.9,2915.7,2918.6,1.243996e+07,buy
997,2022-03-03 21:30:00,2918.7,2922.4,2883.2,2891.8,4.045452e+07,buy


### Hull

In [10]:
def add_hull(ohlcv_df):
    def cal_wma(series):
        weight_list = list(range(1, len(series) + 1))
        weighted_average_list = [i * j for i, j in zip(series, weight_list)]

        wma = sum(weighted_average_list) / sum(weight_list)

        return wma
    
    
    def add_wma(df, input_col, windows):
        wma_list = [None] * (windows - 1)

        for i in range(len(df) - (windows - 1)):
            close_series = df.loc[i:i + (windows - 1), input_col]
            wma = cal_wma(close_series)
            wma_list.append(wma)

        return wma_list
    
    
    def get_hull_side(ohlcv_df):
        if ohlcv_df['hull'] > ohlcv_df['hull_prev']:
            signal_side = 'buy'
        else:
            signal_side = 'sell'

        return signal_side
    
    
    signal_dict = {
        'windows': 180,
        'revert': False
    }

    windows = signal_dict['windows']
    
    temp_df = ohlcv_df.copy()

    temp_df['hwma'] = add_wma(temp_df, 'close', int(round(windows / 2)))
    temp_df['wma'] = add_wma(temp_df, 'close', windows)
    temp_df['twma'] = (2 * temp_df['hwma']) - temp_df['wma']
    temp_df['hull'] = add_wma(temp_df, 'twma', int(round(windows ** (1 / 2))))
    temp_df['hull_prev'] = temp_df['hull'].shift(periods=2)
    temp_df['hull_side'] = temp_df.apply(get_hull_side, axis=1)
    
    ohlcv_df['hull_side'] = temp_df['hull_side']
    
    return ohlcv_df

In [11]:
test_df = add_hull(ohlcv_df)

In [12]:
test_df

,time,open,high,low,close,volume,donchian,hull_side
0,2022-02-21 12:15:00,2740.5,2756.7,2738.1,2747.9,1.322385e+07,NaN,sell
1,2022-02-21 12:30:00,2747.9,2748.6,2738.8,2740.3,1.196507e+07,NaN,sell
2,2022-02-21 12:45:00,2740.3,2743.1,2734.8,2738.0,9.892660e+06,NaN,sell
3,2022-02-21 13:00:00,2738.0,2741.1,2728.7,2736.8,1.480523e+07,NaN,sell
4,2022-02-21 13:15:00,2736.8,2741.4,2734.3,2741.4,3.978329e+06,NaN,sell
...,...,...,...,...,...,...,...,...
994,2022-03-03 20:45:00,2924.5,2932.1,2917.2,2930.6,2.687443e+07,buy,sell
995,2022-03-03 21:00:00,2930.6,2935.0,2919.2,2924.3,3.722915e+07,buy,sell
996,2022-03-03 21:15:00,2924.3,2925.9,2915.7,2918.6,1.243996e+07,buy,sell
997,2022-03-03 21:30:00,2918.7,2922.4,2883.2,2891.8,4.045452e+07,buy,sell
